# Nome: Laura Souza de Carvalho RM: 556320
# Nome: Ali Andrea Mamani Molle RM: 558052

# Resolução enunciado #01

# Recursão é quando uma função "confia" nela mesma pra resolver um problema. Em vez de resolver tudo de uma vez, ela vai dividindo em partes menores e chamando ela mesma pra resolver essas partes.

# É uma técnica onde salva os resultados das chamadas anteriores da função, pra não ter que repetir trabalho. Isso ajuda muito a deixar o programa mais rápido, especialmente em problemas com muitas chamadas repetidas.

# Estrutura de Grafos é uma forma de representar situações em que elementos estão ligados entre si. Ele é formado por "pontos", que representam elementos (como cidades, tarefas ou pessoas), e "ligações", que mostram como esses pontos se conectam.

In [1]:
"""
SISTEMA DE ROTEAMENTO DO METRÔ DE LONDRES
 
Funcionalidades:
1. Calcula rotas mais rápidas, mais longas ou intermediárias
2. Considera tempos de viagem, espera e trocas de linha
3. Visualização interativa dos trajetos com Folium
4. Implementação eficiente com programação dinâmica, recursão e memoization
"""
 
# =============================================================================
# BIBLIOTECA NECESSÁRIA
# =============================================================================
import folium  # Para geração de mapas interativos
 
# =============================================================================
# IMPLEMENTAÇÃO MANUAL DO LRU_CACHE COM cache_clear()
# =============================================================================
class lru_cache:
    def __init__(self, maxsize=None):
        self.maxsize = maxsize
        self.cache = {}
        self.order = []
       
    def cache_clear(self):
        """Limpa o cache"""
        self.cache = {}
        self.order = []
 
    def __call__(self, func):
        def wrapper(*args, **kwargs):
            key = (args, frozenset(kwargs.items()))
           
            if key in self.cache:
                self.order.remove(key)
                self.order.append(key)
                return self.cache[key]
           
            result = func(*args, **kwargs)
           
            if self.maxsize and len(self.cache) >= self.maxsize:
                oldest = self.order.pop(0)
                del self.cache[oldest]
           
            self.cache[key] = result
            self.order.append(key)
            return result
       
        wrapper.cache_clear = self.cache_clear  # Adiciona método à função decorada
        return wrapper
 
# Funções matemáticas manuais (substituem 'math')
def sqrt(x):
    """Calcula raiz quadrada usando operador **"""
    return x**0.5
 
def sin(x):
    """Aproximação de seno usando série de Taylor (4 termos)"""
    return x - (x**3)/6 + (x**5)/120 - (x**7)/5040
 
def cos(x):
    """Aproximação de cosseno usando série de Taylor (4 termos)"""
    return 1 - (x**2)/2 + (x**4)/24 - (x**6)/720
 
def asin(x):
    """Aproximação de arco seno usando série de Taylor (3 termos)"""
    return x + (x**3)/6 + (3*x**5)/40
 
def radians(d):
    """Converte graus para radianos (π ≈ 3.141592653589793)"""
    return d * (3.141592653589793 / 180)
 
# =============================================================================
# CONSTANTES GLOBAIS
# =============================================================================
VELOCIDADE_TREM = 35  # Velocidade média dos trens em km/h
RAIO_TERRA = 6371     # Raio médio da Terra em km
 
# =============================================================================
# CLASSE PRINCIPAL - SISTEMA DE METRÔ
# =============================================================================
class MetroSystem:
    """
    Classe principal que representa a rede de metrô e suas operações
    """
    def __init__(self, stations_coordinates, metro_edges):
        """
        Inicializa o sistema com:
        - stations_coordinates: Lista de tuplas (nome_estação, latitude, longitude)
        - metro_edges: Lista de tuplas (estação1, estação2, linha)
        """
        # Dicionário de estações: {nome: (lat, lon)}
        self.stations = {name: (lat, lon) for name, lat, lon in stations_coordinates}
       
        # Grafo de conexões: {estação: {vizinho: linha}}
        self.graph = self._build_graph(metro_edges)
       
        # Linhas por estação: {estação: [linhas]}
        self.lines = self._extract_lines(metro_edges)
   
    def _build_graph(self, edges):
        """
        Constrói o grafo de conexões entre estações
        Retorna: dicionário de adjacências {estação: {vizinho: linha}}
        """
        graph = {}
        for a, b, line in edges:
            # Adiciona conexão em ambas direções (grafo não-direcionado)
            graph.setdefault(a, {})[b] = line
            graph.setdefault(b, {})[a] = line
        return graph
   
    def _extract_lines(self, edges):
        """
        Mapeia cada estação para suas linhas disponíveis
        Retorna: dicionário {estação: [linhas]}
        """
        lines = {}
        for a, b, line in edges:
            # Adiciona linha às estações (evitando duplicatas)
            if a not in lines:
                lines[a] = []
            if line not in lines[a]:
                lines[a].append(line)
               
            if b not in lines:
                lines[b] = []
            if line not in lines[b]:
                lines[b].append(line)
        return lines
 
    # =========================================================================
    # CÁLCULOS GEOGRÁFICOS E DE TEMPO
    # =========================================================================
    def haversine(self, s1, s2):
        """
        Calcula distância entre duas estações usando fórmula de Haversine
        Fórmula: a = sin²(Δφ/2) + cos(φ1) * cos(φ2) * sin²(Δλ/2)
                 c = 2 * atan2(√a, √(1−a))
                 d = R * c
        Onde φ é latitude, λ é longitude, R é raio da Terra
        Retorna distância em km
        """
        lat1, lon1 = self.stations[s1]
        lat2, lon2 = self.stations[s2]
       
        # Converte graus para radianos
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
       
        # Diferenças das coordenadas
        dlat = lat2 - lat1
        dlon = lon2 - lon1
       
        # Aplica fórmula de Haversine
        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
        return RAIO_TERRA * 2 * asin(sqrt(a))
   
    def travel_time(self, s1, s2):
        """
        Calcula tempo de viagem entre duas estações
        Retorna tempo em minutos
        """
        distance = self.haversine(s1, s2)  # Distância em km
        hours = distance / VELOCIDADE_TREM  # Tempo em horas
        return hours * 60  # Converte para minutos
   
    def waiting_time(self, hora_str):
        """
        Calcula tempo de espera na estação baseado no horário
        Formato de entrada: "HH:MM"
        Retorna tempo em minutos
        """
        h, m = map(int, hora_str.split(':'))
       
        # Manhã (antes das 11h): 1.5 min
        if h < 11 or (h == 11 and m == 0):
            return 1.5
        # Noite (após 18h): 2.0 min
        elif h > 18 or (h == 18 and m > 0):
            return 2.0
        # Tarde (11h-18h): 1.0 min
        else:
            return 1.0
   
    def line_change_penalty(self, current, next_line):
        """
        Calcula penalidade por troca de linha
        Retorna tempo adicional em minutos
        """
        # Sem penalidade se:
        # - Primeira estação (current é None)
        # - Mesma linha
        if not current or current == next_line:
            return 0
        # Penalidade = tempo de espera médio (12:00 está no período de 1 minuto)
        return self.waiting_time("12:00")
 
    # =========================================================================
    # ALGORITMO DE BUSCA DE CAMINHOS
    # =========================================================================
    @lru_cache(maxsize=None)
    def find_paths(self, origem, destino, current_line, visited, hora_inicio, modo):
        """
        Busca todos os caminhos possíveis usando DFS com memoization
        Parâmetros:
        - origem: estação inicial
        - destino: estação final
        - current_line: linha atual (None para primeira estação)
        - visited: tupla de estações já visitadas
        - hora_inicio: horário de partida ("HH:MM")
        - modo: não usado diretamente, mas mantido para consistência
       
        Retorna lista de tuplas (tempo_total, caminho)
        """
        # Caso base: chegou ao destino
        if origem == destino:
            return [(0, [(destino, current_line)])]
       
        results = []
        # Converte visited para tupla imutável (necessário para memoization)
        visited = tuple(sorted(visited + (origem,)))
       
        # Explora todos os vizinhos não visitados
        for vizinho, linha in self.graph[origem].items():
            if vizinho not in visited:
                # Calcula tempo deste segmento
                travel = self.travel_time(origem, vizinho)  # Tempo de viagem
                wait = self.waiting_time(hora_inicio)      # Tempo de espera
                penalty = self.line_change_penalty(current_line, linha)  # Troca de linha
                tempo_segmento = travel + wait + penalty
               
                # Chamada recursiva para continuar o caminho
                for tempo_total, caminho in self.find_paths(
                    vizinho, destino, linha, visited, hora_inicio, modo
                ):
                    # Tempo total = tempo deste segmento + tempo dos segmentos seguintes
                    new_time = tempo_segmento + tempo_total
                   
                    # Constrói caminho (trata caso da primeira estação sem linha definida)
                    new_path = [(origem, current_line or linha)] + caminho
                   
                    results.append((new_time, new_path))
        return results
 
    # =========================================================================
    # FUNÇÃO PRINCIPAL DE INTERFACE
    # =========================================================================
    def menor_tempo_dp(self, origem, destino, hora_inicio, modo='menor'):
        """
        Interface principal para cálculo de rotas
        Parâmetros:
        - origem: estação inicial
        - destino: estação final
        - hora_inicio: horário de partida ("HH:MM")
        - modo: 'menor' (mais rápido), 'maior' (mais longo) ou 'medio' (intermediário)
       
        Retorna dicionário com:
        - Tempo total
        - Sequência de estações
        - Linhas utilizadas
        - Número de trocas
        - Caminho bruto para visualização
        """
        # Limpa cache entre chamadas para evitar interferências
        self.find_paths.cache_clear()
       
        # Busca todos os caminhos possíveis
        caminhos = self.find_paths(origem, destino, None, tuple(), hora_inicio, modo)
       
        if not caminhos:
            return {"error": "Nenhum caminho encontrado"}
       
        # Seleciona caminho baseado no modo
        if modo == 'menor':
            melhor = min(caminhos, key=lambda x: x[0])  # Menor tempo
        elif modo == 'maior':
            melhor = max(caminhos, key=lambda x: x[0])  # Maior tempo
        else:  # modo 'medio'
            # Ordena todos os caminhos por tempo
            caminhos_ordenados = sorted(caminhos, key=lambda x: x[0])
            # Pega o caminho do meio (mediana)
            melhor = caminhos_ordenados[len(caminhos_ordenados)//2]
       
        tempo, caminho = melhor
       
        # Processa informações do caminho
        estacoes = [p[0] for p in caminho]  # Lista de estações
        linhas = []  # Lista de linhas
        trocas = 0   # Contador de trocas
        linha_atual = None  # Linha atual
       
        for _, linha in caminho:
            if linha:
                if linha_atual is None:
                    linha_atual = linha
                elif linha != linha_atual:
                    # Houve troca de linha
                    trocas += 1
                    linha_atual = linha
            linhas.append(linha_atual or "")  # Trata None na primeira estação
       
        return {
            "Tempo total": round(tempo, 1),
            "Caminho": " → ".join(estacoes),
            "Linhas": " → ".join(linhas),
            "Trocas de linha": trocas,
            "raw_path": caminho  # Caminho completo para visualização
        }
 
    # =========================================================================
    # VISUALIZAÇÃO COM FOLIUM
    # =========================================================================
    def plot_path(self, path, output_file="metro_path.html"):
        """
        Gera mapa interativo com o trajeto usando Folium
        Parâmetros:
        - path: lista de tuplas (estação, linha) como retornado por menor_tempo_dp
        - output_file: nome do arquivo HTML para salvar o mapa
       
        Retorna objeto mapa do Folium
        """
        if not path:
            print("Caminho vazio - nada para plotar")
            return
       
        # Cria mapa centrado na primeira estação
        m = folium.Map(location=self.stations[path[0][0]], zoom_start=13)
       
        # Paleta de cores para linhas
        cores_linhas = {
            "Victoria": "blue",
            "Northern": "black",
            "Bakerloo": "brown",
            "Jubilee": "gray",
            "Circle": "yellow",
            "Central": "red",
            "District": "green",
            "Metropolitan": "purple",
            "default": "orange"  # Cor padrão para linhas não mapeadas
        }
       
        # Adiciona marcadores para cada estação
        for i, (estacao, linha) in enumerate(path):
            # Define cor baseada na linha
            cor = cores_linhas.get(linha, cores_linhas["default"])
           
            # Define ícone especial para origem/destino
            if i == 0:  # Origem
                icon = folium.Icon(color='green', icon='home', prefix='fa')
            elif i == len(path)-1:  # Destino
                icon = folium.Icon(color='red', icon='flag', prefix='fa')
            else:  # Estações intermediárias
                icon = folium.Icon(color=cor, icon='subway', prefix='fa')
           
            # Adiciona marcador ao mapa
            folium.Marker(
                location=self.stations[estacao],
                popup=f"<b>{estacao}</b><br>Linha: {linha}",
                icon=icon
            ).add_to(m)
       
        # Adiciona linha conectando as estações
        folium.PolyLine(
            locations=[self.stations[estacao] for estacao, _ in path],
            color='blue',  # Cor da linha no mapa
            weight=2.5,    # Espessura da linha
            opacity=0.8    # Transparência
        ).add_to(m)
       
        # Salva em arquivo HTML
        m.save(output_file)
       
        # Exibe o mapa diretamente no notebook (se estiver no Jupyter)
        display(m)
        return m
 
# =============================================================================
# EXEMPLO DE USO COM AS NOVAS ESTAÇÕES E CONEXÕES
# =============================================================================
if __name__ == "__main__":
    #Dados das estações
    estacoes = [
        ("King's Cross", 51.5308, -0.1238),
        ("Oxford Circus", 51.5154, -0.1410),
        ("Green Park", 51.5067, -0.1428),
        ("Victoria Station", 51.4965, -0.1447),
        ("Euston", 51.5281, -0.1337),
        ("Baker Street", 51.5231, -0.1569),
        ("Paddington", 51.5150, -0.1750),
        ("Bond Street", 51.5145, -0.1494)
    ]
    #Dados das conexões
    conexoes = [
        ("King's Cross", "Oxford Circus", "Victoria"),
        ("Oxford Circus", "Green Park", "Victoria"),
        ("Green Park", "Victoria Station", "Victoria"),
        ("King's Cross", "Euston", "Northern"),
        ("Euston", "Victoria Station", "Northern"),
        ("Baker Street", "Oxford Circus", "Bakerloo"),
        ("Paddington", "Baker Street", "Bakerloo"),
        ("Bond Street", "Green Park", "Jubilee"),
        ("Oxford Circus", "Bond Street", "Jubilee"),
        ("Bond Street", "Bond Street", "Circle")
    ]
    #Resultados: CANINHO MAIS RÁPIDO, LONGO E MÉDIO
    print("Inicializando sistema de metrô com as novas estações...")
    metro = MetroSystem(estacoes, conexoes)
   
    print("\n1. Calculando caminho mais rápido de King's Cross para Victoria Station às 8:30...")
    resultado_rapido = metro.menor_tempo_dp("King's Cross", "Victoria Station", "08:30", "menor")
    print(f"\n--- CAMINHO MAIS RÁPIDO ---")
    print(f"Tempo total: {resultado_rapido['Tempo total']} minutos")
    print(f"Rota: {resultado_rapido['Caminho']}")
    print(f"Linhas: {resultado_rapido['Linhas']}")
    print(f"Trocas: {resultado_rapido['Trocas de linha']}")
   
    print("\nGerando mapa do caminho rápido...")
    metro.plot_path(resultado_rapido['raw_path'], "caminho_rapido.html")
   
    print("\n2. Calculando caminho com tempo médio de King's Cross para Victoria Station às 18:30...")
    resultado_medio = metro.menor_tempo_dp("King's Cross", "Victoria Station", "18:30", "medio")
    print(f"\n--- CAMINHO COM TEMPO MÉDIO ---")
    print(f"Tempo total: {resultado_medio['Tempo total']} minutos")
    print(f"Rota: {resultado_medio['Caminho']}")
    print(f"Linhas: {resultado_medio['Linhas']}")
    print(f"Trocas: {resultado_medio['Trocas de linha']}")
   
    print("\nGerando mapa do caminho médio...")
    metro.plot_path(resultado_medio['raw_path'], "caminho_medio.html")

    print("\n3. Calculando caminho mais longo de King's Cross para Victoria Station às 15:00...")
    resultado_longo = metro.menor_tempo_dp("King's Cross", "Victoria Station", "15:00", "maior")
    print(f"\n--- CAMINHO MAIS LONGO ---")
    print(f"Tempo total: {resultado_longo['Tempo total']} minutos")
    print(f"Rota: {resultado_longo['Caminho']}")
    print(f"Linhas: {resultado_longo['Linhas']}")
    print(f"Trocas: {resultado_longo['Trocas de linha']}")
   
    print("\nGerando mapa do caminho longo...")
    metro.plot_path(resultado_longo['raw_path'], "caminho_longo.html")

Inicializando sistema de metrô com as novas estações...

1. Calculando caminho mais rápido de King's Cross para Victoria Station às 8:30...

--- CAMINHO MAIS RÁPIDO ---
Tempo total: 10.4 minutos
Rota: King's Cross → Euston → Victoria Station
Linhas: Northern → Northern → Northern
Trocas: 0

Gerando mapa do caminho rápido...



2. Calculando caminho com tempo médio de King's Cross para Victoria Station às 18:30...

--- CAMINHO COM TEMPO MÉDIO ---
Tempo total: 13.2 minutos
Rota: King's Cross → Oxford Circus → Green Park → Victoria Station
Linhas: Victoria → Victoria → Victoria → Victoria
Trocas: 0

Gerando mapa do caminho médio...



3. Calculando caminho mais longo de King's Cross para Victoria Station às 15:00...

--- CAMINHO MAIS LONGO ---
Tempo total: 14.2 minutos
Rota: King's Cross → Oxford Circus → Bond Street → Green Park → Victoria Station
Linhas: Victoria → Victoria → Jubilee → Jubilee → Victoria
Trocas: 2

Gerando mapa do caminho longo...
